In [ ]:
import time


class Timer:
    def __init__(self):
        return

    def __enter__(self):
        self._t0 = time.perf_counter()
        return

    def __exit__(self, type, value, traceback):
        t1 = time.perf_counter()
        print(f"Elapsed: {t1-self._t0:0.3f} seconds.")
        return

    pass

In [ ]:
from fmu.sumo.explorer import Explorer

exp = Explorer(env="preview")

In [ ]:
def total_blob_size(sc):
    tbs = sc.metrics.sum("file.size_bytes")
    if tbs == 0:
        tbs = sc.metrics.sum("_sumo.blob_size")
    return tbs


def do_aggregate(tagname, rels, columns):
    gb = 1024*1024*1024
    stats = rels.metrics.stats("_sumo.blob_size")
    minsize, maxsize = stats["min"], stats["max"] 
    print(f"{tagname}: {len(rels)} objects, {len(rels.columns)} columns.")
    print(f"Blob size: min={minsize/gb:.3f} GiB, max={maxsize/gb:.3f} GiB")
    tot_size_bytes = total_blob_size(rels)
    print(f"Total size of input: {tot_size_bytes/gb:.3f} GiB")
    with Timer():
        agg = rels.filter(column=columns)._aggregate(columns=columns)
    print(agg.to_pandas().sort_values(by=["REAL", "DATE"]))


def run_exp(caseuuid, itername, tagname, columns):
    case = exp.get_case_by_uuid(caseuuid)
    print(f"{case.asset}: {case.name}: {caseuuid}")
    rels = case.tables.visible.filter(
        iteration=itername, realization=True, tagname=tagname, column=columns
    )
    do_aggregate(tagname, rels, columns)
    rels=case.tables.hidden.filter(iteration=itername, realization=True, tagname=tagname, column=columns)
    do_aggregate(tagname, rels, columns)
    with Timer():
        agg = case.tables.filter(iteration=itername, aggregation=True, tagname=tagname, column=columns)[0]
        print(agg.to_pandas().sort_values(by=["REAL", "DATE"]))


In [ ]:
run_exp("359e7c72-a4ca-43ee-9203-f09cd0f149a9", "pred-0", "summary", ["FOPT"])

In [ ]:
run_exp("fc6cc7d3-6162-46a3-9d69-48ad1eaecdfb", "iter-0", "summary", ["FOPT"])

In [ ]:
run_exp("5b558daf-61c5-400a-9aa2-c602bb471a16", "iter-0", "summary", ["FOPT"])

In [ ]:
run_exp("cff0e5bf-bef6-48bb-8b25-8c4703f26d87", "iter-0", "summary", ["FOPT", "FOPR"])